In [1]:
import pandas as pd
import numpy as np

In [40]:
d=pd.read_excel('text.xlsx')
#d.set_index('Index',inplace=True)
d.head()

,Index,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,B
0,1,36,27,71,8.1,3.34,11.4,81.5,3243,8.8,42.6,11.7,21,15,59,59,921.870
1,2,35,23,72,11.1,3.14,11.0,78.8,4281,3.6,50.7,14.4,8,10,39,57,997.875
2,3,44,29,74,10.4,3.21,9.8,81.6,4260,0.8,39.4,12.4,6,6,33,54,962.354
3,4,47,45,79,6.5,3.41,11.1,77.5,3125,27.1,50.2,20.6,18,8,24,56,982.291
4,5,43,35,77,7.6,3.44,9.6,84.6,6441,24.4,43.7,14.3,43,38,206,55,1071.289


In [41]:
def normalize_and_add_ones(X):
    X=np.array(X)
    X_max=np.array([[np.max(X[:,column]) 
                   for column in range(X.shape[1])]
                for _ in range(X.shape[0])])
    X_min=np.array([[np.min(X[:,column]) 
                   for column in range(X.shape[1])]
                for _ in range(X.shape[0])])
    X_normalized = (X-X_min)/(X_max-X_min)
    ones=np.array([[1] for _ in range(X_normalized.shape[0])])
    return np.column_stack((ones, X_normalized))

In [42]:
class RidgeRegression:
    def __init__(self):
        return
    def fit(self, X_train, Y_train, LAMBDA):
        assert len(X_train.shape)==2 and X_train.shape[0]==Y_train.shape[0]
        
        W=np.linalg.inv(
          X_train.transpose().dot(X_train)+LAMBDA*np.identity(X_train.shape[1])
        ).dot(X_train. transpose()).dot(Y_train)
        return W
    def predict(self, W, X_new):
        X_new=np.array(X_new)
        Y_new=X_new.dot(W)
        return Y_new
    def compute_RSS(self, Y_new, Y_predicted):
        loss=1./Y_new.shape[0]*np.sum((Y_new-Y_predicted)**2)
        return loss
    def get_the_best_LAMBDA(self, X_train, Y_train):
        def cross_validation(num_folds, LAMBDA):
            row_ids=np.array(range(X_train.shape[0]))
            valid_ids=np.split(row_ids[:len(row_ids)-len(row_ids)%num_folds], num_folds)
            valid_ids[-1]=np.append(valid_ids[-1], row_ids[len(row_ids)-len(row_ids)%num_folds:])
            train_ids=[[k for k in row_ids if k not in valid_ids[i]] for i in range(num_folds)]
            aver_RSS=0
            for i in range(num_folds):
                valid_part={'X':X_train[valid_ids[i]], 'Y':Y_train[valid_ids[i]]}
                train_part={'X':X_train[train_ids[i]], 'Y':Y_train[train_ids[i]]}
                W=self.fit(train_part['X'], train_part['Y'], LAMBDA)
                Y_predicted=self.predict(W, valid_part['X'])
                aver_RSS += self.compute_RSS(valid_part['Y'], Y_predicted)
            return aver_RSS/num_folds
        def range_scan(best_LAMBDA, minimum_RSS, LAMBDA_values):
            for current_LAMBDA in LAMBDA_values:
                aver_RSS = cross_validation(num_folds=5, LAMBDA=current_LAMBDA)
                if aver_RSS<minimum_RSS:
                    best_LAMBDA = current_LAMBDA
                    minimum_RSS=aver_RSS
            return best_LAMBDA, minimum_RSS
        best_LAMBDA, minimum_RSS= range_scan(best_LAMBDA=0, minimum_RSS=1000**2,LAMBDA_values=range(50))
        LAMBDA_values=[k*1./1000 for k in range(max(0,(best_LAMBDA-1)*1000), (best_LAMBDA+1)*1000,1)]
        best_LAMBDA, minimum_RSS= range_scan(best_LAMBDA=best_LAMBDA, minimum_RSS=minimum_RSS, LAMBDA_values=LAMBDA_values)
        return best_LAMBDA 



In [43]:
dd=normalize_and_add_ones(d)

X=dd[0:60,0:16]
#print(X)
Y=dd[0:60,16]
#print(Y)
X_train, Y_train =X[:50], Y[:50]
X_test, Y_test=X[50:],Y[50:]
ridge_regression=RidgeRegression()
best_LAMBDA=ridge_regression.get_the_best_LAMBDA(X_train, Y_train)
print ('Best LAMBDA:', best_LAMBDA)

W_learned=ridge_regression.fit(X_train=X_train, Y_train=Y_train, LAMBDA=best_LAMBDA)
Y_predicted=ridge_regression.predict(W=W_learned, X_new=X_test)
print (ridge_regression.compute_RSS(Y_new=Y_test, Y_predicted=Y_predicted))


Best LAMBDA: 10.21
0.014649450130075226
